In [2]:
import numpy as np

# Binary numbers

## 1. Practice writing numbers in binary format, i.e. in base-2, i.e. as a bit string.
1) Write the numbers from 0₁₀ to 15₁₀ in binary.

| Decimal |   0  |   1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12  |  13  |  14  |  15  |
|  ---:   | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |  
|  Binary | 0000 | 0001 | 0010 | 0011 | 0100 | 0101 | 0110 | 0111 | 1000 | 1001 | 1010 | 1011 | 1100 | 1101 | 1110 | 1111 |


2) Convert these binary numbers to decimal numbers:
11010₂ , 1010101₂ , 10000000000₂

|  Binary |   110110  |   1010101  |   10000000000  |
|  ---:   | :-------: | :--------: | :-------------: |
| Decimal | 32+16+4+2 |  64+16+4+1 |            1024 |
|         |        54 |  85        |            1024 |

3) Convert these decimal numbers to binary numbers:
77₁₀ , 511₁₀ , 513₁₀

| Decimal |        77 |  511                        |         513 |
|  ---:   | :-------: | :------------------------:  | :---------: |
| Decimal | 64+8+4+1  |  256+128+64+32+16+8+4+2+1   | 521+1       |
|  Binary |   1001101  |   111111111                |  1000000001 |

4) How many bits (binary digits) are needed to represent these numbers?
30₁₀ , 300₁₀ , 3000₁₀ , 30000₁₀

|  Decimal |   30      |  300        |    3000    | 30000  |
|  ----:   | :-------: | :--------:  | :--------: | :----: |
| num bits |   5       | 9           |  12        | 15     |

$N_\mathrm{bits} = \lfloor\log_2(x) + 1\rfloor$

In [ ]:
# 0b110110, 0b1010101, 0b10000000000
# 0b1001101, 0b111111111, 0b1000000001

np.float64(15.0)

### 2. Binary addition. Calculate the following

1. 1101₂ + 0010₂ = 1111₂
2. 1101₂ + 0011₂ = 10000₂
3. 111111₂ + 000100₂ = 1000011₂

### 3. The bitwise addition modulo 2 or XOR (exlusive-OR) with the symbol can also be considered as addition without carry or as bit value toggling with a mask.

1. 1101₂ ⊕ 10₂ = 1111
2. 1101₂ ⊕ 11₂ = 1110
3. 101101001₂ ⊕ 111001011₂ = 010100010


## 4. Division by factors of 2.
1. 100₂ / 10₂ = 010
2. 1100₂ / 10₂ = ?
3. 1000₂ / 1000₂ = ?


In [87]:
0x9f

159